In [5]:
import pandas as pd
import numpy as np
from scipy.stats import t, norm

# 加载数据
data = pd.read_csv('problem5.csv')
# 检查数据结构
print(data.head())


            A          B           C          D
0   99.748060  42.659947  155.181358  66.545111
1   99.452137  41.392601  146.732762  63.684575
2  102.867505  42.900677  146.417105  64.468899
3  105.524561  42.651742  146.989262  61.148250
4  105.305369  42.904467  145.434970  60.744525


a. Calculate ES (5%) for each asset

In [6]:
# 5a. 计算每个资产的 5% 条件风险值 (ES)

def fit_t_and_calculate_es(asset_returns, confidence_level=0.05):
    """
    拟合广义 t 分布并计算条件风险值 (ES)
    """
    # 拟合广义 t 分布
    params = t.fit(asset_returns)
    df, loc, scale = params

    # 计算 5% VaR
    var_5 = t.ppf(confidence_level, df, loc, scale)

    # 计算 5% ES
    es_5 = t.expect(lambda x: x, args=(df,), loc=loc, scale=scale, lb=-np.inf, ub=var_5) / confidence_level
    return es_5

# 为每个资产计算 5% ES
es_results = {}
for column in data.columns:
    asset_returns = data[column].values
    es_results[column] = fit_t_and_calculate_es(asset_returns)

print("每个资产的 5% 条件风险值 (ES):")
print(es_results)


每个资产的 5% 条件风险值 (ES):
{'A': np.float64(96.71628249385503), 'B': np.float64(40.367425425591534), 'C': np.float64(79.12153348845283), 'D': np.float64(58.463470484231856)}


b. Calculate ES (5%) for a portfolio of Asset 1 &2 and a portfolio of Asset 3&4

In [7]:
# 5b. 计算组合资产的 5% 条件风险值 (ES)

def simulate_gaussian_copula(returns, n_samples=10000):
    """
    模拟高斯 Copula 样本
    """
    # 计算相关性矩阵
    corr_matrix = np.corrcoef(returns.T)

    # 生成标准正态分布的样本
    z = np.random.multivariate_normal(mean=np.zeros(corr_matrix.shape[0]), cov=corr_matrix, size=n_samples)

    # 转换为累积概率
    u = norm.cdf(z)
    return u

def calculate_portfolio_es(copula_samples, marginal_distributions, weights, confidence_level=0.05):
    """
    使用高斯 Copula 和边际分布计算组合的条件风险值 (ES)
    """
    # 生成组合收益率
    portfolio_returns = np.zeros(copula_samples.shape[0])
    for i, marginal in enumerate(marginal_distributions):
        # 逆 CDF 将 Copula 样本映射回原始分布
        portfolio_returns += weights[i] * t.ppf(copula_samples[:, i], *marginal)
    
    # 计算组合的 5% VaR
    var_5 = np.percentile(portfolio_returns, confidence_level * 100)

    # 计算组合的 5% ES
    es_5 = portfolio_returns[portfolio_returns <= var_5].mean()
    return es_5

# 资产 1 和 2 的组合
returns_12 = data.iloc[:, [0, 1]].values
copula_samples_12 = simulate_gaussian_copula(returns_12)
marginal_12 = [t.fit(data.iloc[:, i].values) for i in [0, 1]]
weights_12 = [0.5, 0.5]  # 假设等权重
es_12 = calculate_portfolio_es(copula_samples_12, marginal_12, weights_12)

print(f"组合 1 (资产 1 和 2) 的 5% 条件风险值 (ES): {es_12}")

# 资产 3 和 4 的组合
returns_34 = data.iloc[:, [2, 3]].values
copula_samples_34 = simulate_gaussian_copula(returns_34)
marginal_34 = [t.fit(data.iloc[:, i].values) for i in [2, 3]]
weights_34 = [0.5, 0.5]  # 假设等权重
es_34 = calculate_portfolio_es(copula_samples_34, marginal_34, weights_34)

print(f"组合 2 (资产 3 和 4) 的 5% 条件风险值 (ES): {es_34}")


组合 1 (资产 1 和 2) 的 5% 条件风险值 (ES): 69.23810078640564
组合 2 (资产 3 和 4) 的 5% 条件风险值 (ES): 75.10422925297601


c. Calculate ES (5%) for a portfolio of all 4 assets.

In [8]:
# 5c. 计算包含所有资产的组合的 5% 条件风险值 (ES)

# 获取所有资产的返回数据
returns_all = data.values
copula_samples_all = simulate_gaussian_copula(returns_all)
marginal_all = [t.fit(data.iloc[:, i].values) for i in range(data.shape[1])]
weights_all = [1 / data.shape[1]] * data.shape[1]  # 假设等权重

es_all = calculate_portfolio_es(copula_samples_all, marginal_all, weights_all)

print(f"包含所有资产的组合的 5% 条件风险值 (ES): {es_all}")


包含所有资产的组合的 5% 条件风险值 (ES): 76.42802579344664
